# *SQL e Python no Mundo do Futebol e suas possíveis aplicações*

## <u>Com Python...<u/>

Para o  Algoritmo de regressão logística simulei uma situação onde conto com um dataset com diversas informações classificando as pessoas que recebem  mais ou menos que 25 mil reais anuais, sendo essas pessoas, sócios torcedores de um time x. O objetivo é funcionar como uma empresa que vende relatórios de inteligência para que possam ser aproveitados pelos clubes de futebol.

##### O que é regressão logística?
A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, em função de uma ou mais variáveis independentes contínuas e/ou binárias.

![](Images/reg_log1.JPG)

#### Importando todas as bibliotecas necessárias

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import joblib

%matplotlib inline
mpl.rcParams['figure.dpi']= 100

#### Leitura dos dataframes

In [5]:
df = pd.read_csv('adult2.csv',names=[
        "Idade", "Classe de Trabalho", "Peso Final", "Educacao", "Id_Educacao", "Estado Civil",
        "Ocupacao", "Relacao", "Raca", "Sexo", "Ganho de Capital", "Perda de Capital",
        "Horas-Por-Semana", "Nacionalidade", "Ganho Anual"], # Traduzindo colunas para o português
        sep=r'\s*;\s*',
        engine='python',
        na_values="?", skiprows = 1).dropna() # Linha de código feita para excluir linhas que contém dados faltantes
df

,Idade,Classe de Trabalho,Peso Final,Educacao,Id_Educacao,Estado Civil,Ocupacao,Relacao,Raca,Sexo,Ganho de Capital,Perda de Capital,Horas-Por-Semana,Nacionalidade,Ganho Anual
0,50,Trabalhador-nao-autonomo,83311,Bacharelado,13,Conjuge-civ-casado,Executivo-gerencial,Marido,Branco,Masculino,0,0,13,Brasil,<=25K
1,38,Privado,215646,Formado-EM,9,Divorciado,Limpeza,Nao-membro-da-familia,Branco,Masculino,0,0,40,Brasil,<=25K
2,53,Privado,234721,2-colegial,7,Conjuge-civ-casado,Limpeza,Marido,Negro,Masculino,0,0,40,Brasil,<=25K
3,28,Privado,338409,Bacharelado,13,Conjuge-civ-casado,Profissional-especializado,Esposa,Negro,Feminino,0,0,40,Brasil,<=25K
4,37,Privado,284582,Mestrado,14,Conjuge-civ-casado,Executivo-gerencial,Esposa,Branco,Feminino,0,0,40,Brasil,<=25K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Privado,257302,Associacao-academica,12,Conjuge-civ-casado,Suporte-Tecnico,Esposa,Branco,Feminino,0,0,38,Brasil,<=25K
32556,40,Privado,154374,Formado-EM,9,Conjuge-civ-casado,Operador-maquina,Marido,Branco,Masculino,0,0,40,Brasil,>25K
32557,58,Privado,151910,Formado-EM,9,Viuvo,Assistente-administrativo,Nao-casado,Branco,Feminino,0,0,40,Brasil,<=25K
32558,22,Privado,201490,Formado-EM,9,Nunca-casou,Assistente-administrativo,Filho-proprio,Branco,Masculino,0,0,20,Brasil,<=25K


#### Substituindo df por df_novo

In [6]:
df_novo = df

### Transformando as 'str' do dataset em números para o treinamento dos algoritmos

In [7]:
# Função 'embar_atrib'
def embar_atrib(atributo, alvo):
    dic_em_branco = {}
    localTargetData = {}
    for index, element in enumerate(atributo):
        if element not in localTargetData:
            if alvo[index] == '<=25K':
                localTargetData[element] = [1,0]
            else:
                localTargetData[element] = [0,1]
        else:
            if alvo[index] == '<=25K':
                localTargetData[element][0] +=1
            else:
                localTargetData[element][1] += 1
    return localTargetData

In [8]:
# Laço for responsável por fazer as trocas de 'str' por 'float' nos df's
for i in (1,3,5,6,7,8,9,13):
    replaceDict = embar_atrib(df_novo.iloc[:,i:i+1].values.transpose()[0].tolist(),df_novo.iloc[:,df_novo.shape[1]-1:df_novo.shape[1]].values.transpose()[0].tolist())
    for key in replaceDict:
        df_novo = df_novo.replace(key, replaceDict[key][1]/(replaceDict[key][1]+replaceDict[key][0]))
        print('Trocando {0} por {1}'.format(key, replaceDict[key][1]/(replaceDict[key][1]+replaceDict[key][0]) ))

Trocando Trabalhador-nao-autonomo por 0.2849271940181031
Trocando Privado por 0.21867289390200917
Trocando Chefe-de-estado por 0.27216653816499614
Trocando Governo-federal por 0.38645833333333335
Trocando Governo-municipal por 0.29479216435738176
Trocando Trabalhador-autonomo por 0.557347670250896
Trocando Sem-remuneracao por 0.0
Trocando Bacharelado por 0.4199961397413627
Trocando Formado-EM por 0.16342295345104335
Trocando 2-colegial por 0.056818181818181816
Trocando Mestrado por 0.5617910447761194
Trocando 9-ano por 0.056155507559395246
Trocando Faculdade por 0.19955719557195573
Trocando Associacao-academica por 0.25392156862745097
Trocando Curso-profissionalizante por 0.2634367903103709
Trocando 7ou8-ano por 0.06631762652705062
Trocando Doutorado por 0.7412060301507538
Trocando Pos-graduado por 0.7437275985663082
Trocando 5ou6-ano por 0.0462046204620462
Trocando 1-colegial por 0.07220216606498195
Trocando Pre-escola por 0.0
Trocando 3-colegial por 0.07888040712468193
Trocando 1ou4-

#### Substituindo as classes finais (>25K por 1; <=25K por 0) 

In [9]:
df_novo = df_novo.replace('>25K', 1)
df_novo = df_novo.replace('<=25K',0)

### Novo dataset

In [10]:
df_novo

,Idade,Classe de Trabalho,Peso Final,Educacao,Id_Educacao,Estado Civil,Ocupacao,Relacao,Raca,Sexo,Ganho de Capital,Perda de Capital,Horas-Por-Semana,Nacionalidade,Ganho Anual
0,50,0.284927,83311,0.419996,13,0.454774,0.484014,0.455683,0.264030,0.313802,0,0,13,0.249048,0
1,38,0.218673,215646,0.163423,9,0.107327,0.062774,0.106562,0.264030,0.313802,0,0,40,0.249048,0
2,53,0.218673,234721,0.056818,7,0.454774,0.062774,0.455683,0.129942,0.313802,0,0,40,0.249048,0
3,28,0.218673,338409,0.419996,13,0.454774,0.449034,0.491289,0.129942,0.113494,0,0,40,0.249048,0
4,37,0.218673,284582,0.561791,14,0.454774,0.484014,0.491289,0.264030,0.113494,0,0,40,0.249048,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,0.218673,257302,0.253922,12,0.454774,0.304957,0.491289,0.264030,0.113494,0,0,38,0.249048,0
32556,40,0.218673,154374,0.163423,9,0.454774,0.124875,0.455683,0.264030,0.313802,0,0,40,0.249048,1
32557,58,0.218673,151910,0.163423,9,0.096429,0.134518,0.066035,0.264030,0.113494,0,0,40,0.249048,0
32558,22,0.218673,201490,0.163423,9,0.048633,0.134518,0.014586,0.264030,0.313802,0,0,20,0.249048,0


### Aplicando o algoritmo de Regressão Logística

In [11]:
# A função to_numpy() é usada para retornar um ndarray NumPy que representa os valores em determinada série ou índice.
df_novo1 = df_novo.to_numpy()
df_novo1

array([[5.00000000e+01, 2.84927194e-01, 8.33110000e+04, ...,
        1.30000000e+01, 2.49047759e-01, 0.00000000e+00],
       [3.80000000e+01, 2.18672894e-01, 2.15646000e+05, ...,
        4.00000000e+01, 2.49047759e-01, 0.00000000e+00],
       [5.30000000e+01, 2.18672894e-01, 2.34721000e+05, ...,
        4.00000000e+01, 2.49047759e-01, 0.00000000e+00],
       ...,
       [5.80000000e+01, 2.18672894e-01, 1.51910000e+05, ...,
        4.00000000e+01, 2.49047759e-01, 0.00000000e+00],
       [2.20000000e+01, 2.18672894e-01, 2.01490000e+05, ...,
        2.00000000e+01, 2.49047759e-01, 0.00000000e+00],
       [5.20000000e+01, 5.57347670e-01, 2.87927000e+05, ...,
        4.00000000e+01, 2.49047759e-01, 1.00000000e+00]])

In [12]:
nrow,ncol = df_novo1.shape
df_novo1.shape

(30717, 15)

30717 Linhas e 15 Colunas

In [13]:
# Atributos a serem considerados
X = df_novo1[:,0:ncol-1]
df_novo1[:,0:ncol-1]

array([[5.00000000e+01, 2.84927194e-01, 8.33110000e+04, ...,
        0.00000000e+00, 1.30000000e+01, 2.49047759e-01],
       [3.80000000e+01, 2.18672894e-01, 2.15646000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       [5.30000000e+01, 2.18672894e-01, 2.34721000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       ...,
       [5.80000000e+01, 2.18672894e-01, 1.51910000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       [2.20000000e+01, 2.18672894e-01, 2.01490000e+05, ...,
        0.00000000e+00, 2.00000000e+01, 2.49047759e-01],
       [5.20000000e+01, 5.57347670e-01, 2.87927000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01]])

In [14]:
# Classe final
y = df_novo1[:,-1]
df_novo1[:,-1]

array([0., 0., 0., ..., 0., 0., 1.])

In [15]:
# Padronização de dados
logi = StandardScaler().fit(X)
X = logi.transform(X)
X

array([[ 0.88094485,  0.50614735, -1.01025865, ..., -0.21918304,
        -2.33195294,  0.        ],
       [-0.03381291, -0.42849496,  0.24462059, ..., -0.21918304,
        -0.0792085 ,  0.        ],
       [ 1.10963429, -0.42849496,  0.42550109, ..., -0.21918304,
        -0.0792085 ,  0.        ],
       ...,
       [ 1.49078336, -0.42849496, -0.3597621 , ..., -0.21918304,
        -0.0792085 ,  0.        ],
       [-1.25348993, -0.42849496,  0.11038497, ..., -0.21918304,
        -1.74790808,  0.        ],
       [ 1.03340448,  4.34915378,  0.93003205, ..., -0.21918304,
        -0.0792085 ,  0.        ]])

In [16]:
# Separando os X's e Y's para teste e treino (30% dos dados serão utilizados para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [17]:
# Função do algoritmo de regressão linear
logist = LogisticRegression(solver = 'lbfgs', multi_class = 'auto', max_iter = 1000)

# Normalização dos valores X_train e Y_train com a função 'fit()'
logist.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [18]:
# Valores preditos
y_pred = logist.predict(X_test)
y_pred

array([0., 1., 0., ..., 0., 1., 0.])

In [19]:
# Valores testados
y_test

array([0., 0., 0., ..., 0., 0., 0.])

### Avaliando o algoritmo de Regressão Logística

In [20]:
print(classification_report(y_test, y_pred, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      6972
           1       0.73      0.58      0.65      2244

    accuracy                           0.85      9216
   macro avg       0.80      0.76      0.77      9216
weighted avg       0.84      0.85      0.84      9216



![](Images/metricas.JPG)

In [21]:
confusion_matrix(y_test, y_pred)

array([[6480,  492],
       [ 935, 1309]], dtype=int64)

![](Images/Matriz1.JPG)
![](Images/Matriz2.JPG)

Das 7415 pessoas da amostra que ganham menos de 25K, 6480 foram previstas corretamente, enquanto 935 foram preditas ganhando mais de 25K. Já das 1801 pessoas da amostra que ganham mais de 25K, 1309 foram previstas corretamente, enquanto 492 foram preditas ganhando menos de 25K.

### Persistindo o algoritmo de Regressão Logística

In [22]:
arq_log = 'LOG_SocioTorcedor.sav'
joblib.dump(logist,arq_log)

['LOG_SocioTorcedor.sav']

In [23]:
log_carregado = joblib.load(arq_log)
resultado = log_carregado.score(X_test, y_test)
print(resultado)

0.8451605902777778


#### ACURÁCIA FINAL = 84,5%